In [80]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split data into source and target domains
X_source, X_target, y_source, y_target = train_test_split(X, y, test_size=0.5, random_state=42)


# Convert numpy arrays to PyTorch tensors
X_source = torch.from_numpy(X_source).float()
X_target = torch.from_numpy(X_target).float()
y_source = torch.from_numpy(y_source).long()

# Define the feature extractor
class FeatureExtractor(nn.Module):
    def __init__(self, input_size):
        super(FeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Adjusted input_size to match the number of features
        self.fc2 = nn.Linear(64, 32)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)  # Change input_size to match the output size of the feature extractor
        self.fc2 = nn.Linear(16, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Define the generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    
# Define the task-specific network
class TaskNetwork(nn.Module):
    def __init__(self,input_size):
        super(TaskNetwork, self).__init__()
        self.feature_extractor = FeatureExtractor(input_size)
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 3)  # Output layer for Iris dataset with 3 classes
    
    def forward(self, x):
        features = self.feature_extractor(x)
        x = F.relu(self.fc1(features))
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Initialize the models
feature_extractor = FeatureExtractor(input_size=4)
# Pass the output size of the feature extractor to the discriminator
discriminator = Discriminator(input_size=32)
generator = Generator()
task_network = TaskNetwork(input_size=4)


# Define the adversarial learning process
class AdversarialNetwork(nn.Module):
    def __init__(self, generator, discriminator):
        super(AdversarialNetwork, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
    
    def forward(self, x):
        generated = self.generator(x)
        return self.discriminator(generated)

# Initialize the adversarial network
adversarial_network = AdversarialNetwork(generator, discriminator)

# Define optimizers
optimizer_gen = optim.Adam(generator.parameters(), lr=0.001)
optimizer_disc = optim.Adam(discriminator.parameters(), lr=0.001)
optimizer_task = optim.Adam(task_network.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    # Train discriminator
    optimizer_disc.zero_grad()
    real_target_output = discriminator(feature_extractor(X_target))  # Apply feature extractor to target data
    fake_output = discriminator(generator(X_source))
    loss_discriminator = F.binary_cross_entropy(fake_output, torch.zeros_like(fake_output)) + \
                        F.binary_cross_entropy(real_target_output, torch.ones_like(real_target_output))
    loss_discriminator.backward()
    optimizer_disc.step()

    # Train generator
    optimizer_gen.zero_grad()
    fake_output = discriminator(generator(X_source))
    loss_generator = F.binary_cross_entropy(fake_output, torch.ones_like(fake_output))
    loss_generator.backward()
    optimizer_gen.step()

    # Train task network
    optimizer_task.zero_grad()
    source_features = feature_extractor(X_source)
    predicted = task_network(X_source)
    loss_task = F.cross_entropy(predicted, y_source)
    loss_task.backward()
    optimizer_task.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{epochs}], Discriminator Loss: {loss_discriminator.item()}, '
              f'Generator Loss: {loss_generator.item()}, Task Network Loss: {loss_task.item()}')

Epoch [0/100], Discriminator Loss: 1.4074294567108154, Generator Loss: 0.6290392279624939, Task Network Loss: 1.0926198959350586
Epoch [10/100], Discriminator Loss: 1.3915977478027344, Generator Loss: 0.6163039207458496, Task Network Loss: 1.0637712478637695
Epoch [20/100], Discriminator Loss: 1.367431879043579, Generator Loss: 0.6166011691093445, Task Network Loss: 1.0435699224472046
Epoch [30/100], Discriminator Loss: 1.341942548751831, Generator Loss: 0.6241279244422913, Task Network Loss: 1.0181152820587158
Epoch [40/100], Discriminator Loss: 1.3245271444320679, Generator Loss: 0.63446444272995, Task Network Loss: 0.9937199950218201
Epoch [50/100], Discriminator Loss: 1.309558629989624, Generator Loss: 0.6493899822235107, Task Network Loss: 0.9652581810951233
Epoch [60/100], Discriminator Loss: 1.2380821704864502, Generator Loss: 0.7220100164413452, Task Network Loss: 0.926879346370697
Epoch [70/100], Discriminator Loss: 1.168175458908081, Generator Loss: 0.7951335906982422, Task N

In [81]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Evaluation function
def evaluate_model(X_test, y_test):
    # Make predictions
    task_network.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient tracking
        y_pred_probs = task_network(X_test)
        _, y_pred = torch.max(y_pred_probs, 1)  # Convert probabilities to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test.numpy(), y_pred.numpy())
    precision = precision_score(y_test.numpy(), y_pred.numpy(), average='macro')
    recall = recall_score(y_test.numpy(), y_pred.numpy(), average='macro')
    f1 = f1_score(y_test.numpy(), y_pred.numpy(), average='macro')

    # Calculate confusion matrix
    cm = confusion_matrix(y_test.numpy(), y_pred.numpy())

    return accuracy, precision, recall, f1, cm

# Splitting the test data
X_source_test, X_target_test, y_source_test, y_target_test = train_test_split(X_target, y_target, test_size=0.5, random_state=42)

# Evaluate the model
accuracy, precision, recall, f1, cm = evaluate_model(X_source_test, torch.from_numpy(y_source_test).long())  # Here, we convert y_source_test to PyTorch tensor

# Print evaluation results
print("Evaluation Results:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

Evaluation Results:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[12  0  0]
 [ 0 12  0]
 [ 0  0 13]]
